In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras

In [2]:
housing = fetch_california_housing()

In [3]:
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [4]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)     #get fit parameters for scaling
X_valid = scaler.transform(X_valid)         # scale validation and test test using those parameters
X_test = scaler.transform(X_test)

# To use Grid/Randomized search we need to wrap our Keras Models in objects that mimic regular Scikit-learn Regressors

In [5]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation-"relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizer.SGD(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [6]:
# Create a Keras regressor based on the build_model() function
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)     #KerasRegressor object is a thin wrapper around Keras model

# This model can now be used like a regular Scikit-Learn regressor.


C:\Users\viraj\AppData\Local\Temp\ipykernel_3288\741151798.py:2: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)     #KerasRegressor object is a thin wrapper around Keras model


In [8]:
# Using RandomizedSearch

from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

param_dist = {
    "n_hidden" : [1, 2],
    "n_neurons" : np.arange(10,50),
    "learning_rate" : reciprocal(3e-4, 3e-2),
}